# Importer les librairies, et fichiers .py auxiliaires

In [1]:
import numpy as np
import pandas as pd
from preprocess_data import *

# Jalon 1) Naive de Bayes vanille
K-Fold Validation croisée ($k = 5)$:
- α: Lissage du postérieur de Bayes P[C| X = x_i]